In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np

In [27]:
data1 = pd.read_csv('/home/denis/Hack_t1/ml-part-hackaton/history-Table 1.csv', sep=';', on_bad_lines='skip', engine='python')

In [28]:
data1.head()

,entity_id,history_property_name,history_date,history_version,history_change_type,history_change,Столбец1,Unnamed: 7
0,94297.0,Время решения 3ЛП ФАКТ,9/10/24 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN,NaN
1,94297.0,Время решения (ФАКТ),9/10/24 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN,NaN
2,94297.0,Исполнитель,7/13/23 11:07,1.0,FIELD_CHANGED,user409017mail@mail.com -> user408045mail@mail...,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,94297.0,Исполнитель,7/21/23 11:06,3.0,FIELD_CHANGED,user4080458@mail.com -> user4091341@mail.com,NaN,NaN


In [26]:
data1.shape

(64180, 1)

In [25]:
print(data1.loc(1))

In [29]:
data2 = pd.read_csv('data_for_spb_hakaton_entities1-Table 1.csv', sep=';', on_bad_lines='skip', engine='python')

In [30]:
data2.head()

,entity_id,area,type,status,state,priority,ticket_number,name,create_date,created_by,...,updated_by,parent_ticket_id,assignee,owner,due_date,rank,estimation,spent,workgroup,resolution
0,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,А. К.,72779.0,А. К.,А. К.,NaN,0|qzzywk:,60.0,NaN,NaN,Готово
1,102481,Система.Ошибки,История,Закрыто,Normal,Критический,PPIN-1175,[ГенераторДокументов] Интеграция со Система.Ге...,2023-05-12 13:33:55.918127,А. З.,...,NaN,3488105.0,А. Е.,А. Е.,NaN,0|qv7n1c:y,432000.0,NaN,Новая функциональность,Готово
2,1805925,Система.Таск-трекер,Дефект,Тестирование,Normal,Высокий,PPTS-3189,[FE] История изменений. Пустые строки в истори...,2023-07-12 09:36:04.479760,А. К.,...,А. К.,NaN,А. К.,А. К.,NaN,0|qzsklw:,60.0,NaN,NaN,NaN
3,1934905,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-3383,[FE] Зависимые поля. Тип реакции disable НЕ ра...,2023-08-04 11:32:25.829919,А. Д.,...,А. К.,NaN,А. К.,Я. П.,NaN,0|qzh3e8:,NaN,NaN,NaN,Готово
4,1943849,Система.Ошибки,Дефект,Отклонен исполнителем,Normal,Низкий,PPIN-1609,[BE] При сортировке по Теме если знаки препина...,2023-08-09 06:20:44.391950,А. С.,...,А. Е.,2083371.0,А. М.,А. Ж.,NaN,0|qzzywj:zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz...,NaN,NaN,NaN,Отклонено


In [31]:
data2.columns

Index(['entity_id', 'area', 'type', 'status', 'state', 'priority',
       'ticket_number', 'name', 'create_date', 'created_by', 'update_date',
       'updated_by', 'parent_ticket_id', 'assignee', 'owner', 'due_date',
       'rank', 'estimation', 'spent', 'workgroup', 'resolution'],
      dtype='object')

In [20]:
data3 = pd.read_csv('/home/denis/Hack_t1/ml-part-hackaton/sprints-Table 1.csv', sep=';', on_bad_lines='skip', engine='python')

In [21]:
data3.head()

,sprint_name,sprint_status,sprint_start_date,sprint_end_date,entity_ids
0,Спринт 2024.3.1.NPP Shared Sprint,Закрыт,2024-07-03 19:00:00.000000,2024-07-16 19:00:00.000000,"{4449728,4450628,4451563,4451929,4452033,44522..."
1,Спринт 2024.3.2.NPP Shared Sprint,Закрыт,2024-07-17 19:00:00.000000,2024-07-30 19:00:00.000000,"{4506286,4429413,4327418,4370041,4525683,43267..."
2,Спринт 2024.3.3.NPP Shared Sprint,Закрыт,2024-07-31 19:00:00.000000,2024-08-13 19:00:00.000000,"{4646403,4176602,4555571,4497495,4340795,46178..."
3,Спринт 2024.3.4.NPP Shared Sprint,Закрыт,2024-08-14 19:00:00.000000,2024-08-27 19:00:00.000000,"{4805777,4596004,4594168,4523718,4579094,46853..."
4,Спринт 2024.3.5.NPP Shared Sprint,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,"{4861060,4856927,4868451,4092647,4932565,49838..."
